## Importación de librerías

In [65]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

## Configuración de variables globales

In [ ]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
OUTPUT_CSV = SETUP_JSON['dataset_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['dataset_parquet'] # Output CSV file

## Cargar el dataset

In [54]:
df = pd.read_csv(OUTPUT_CSV) # Leer csv

## Extracción de secuencias de paquetes

Obtener todas las secuencias de paquetes presentes en el dataset.

In [55]:
# Crear una copia del DataFrame original para no modificarlo directamente.
df_temp = df.copy()

# Cada vez que el valor de 'label' cambia, se incrementa el contador acumulativo.
df_temp['sequence'] = (df_temp['Label'] != df_temp['Label'].shift()).cumsum()

# Agrupar el DataFrame por la columna 'sequence' y crear una lista de DataFrames.
sequences = [sequence for _, sequence in df_temp.groupby('sequence')]  

for sequence_df in sequences:  # Iterar sobre cada DataFrame en la lista de secuencias.
    # Eliminar la columna 'sequence' de cada DataFrame, ya que solo se usó para agrupar.
    sequence_df.drop(columns='sequence', inplace=True)  
    # Reiniciar los índices de cada DataFrame para que comiencen desde 0.
    sequence_df.reset_index(drop=True, inplace=True)

## Análisis de las secuencias

In [56]:
print(f"Total de secuencias de paquetes encontradas: {len(sequences)}")

Total de secuencias de paquetes encontradas: 15947


In [57]:
print(f"Primera secuencia: {sequences[0]['Label'].unique()}")
print(f"Número de paquetes de la primera secuencia: {sequences[0].shape[0]}")

Primera secuencia: [0]
Número de paquetes de la primera secuencia: 94


In [58]:
print(f"Última secuencia: {sequences[-1]['Label'].unique()}")
print(f"Número de paquetes de la última secuencia: {sequences[-1].shape[0]}")

Última secuencia: [1]
Número de paquetes de la última secuencia: 17


In [59]:
# Almacenar los tamaños de las secuencias y las secuencias en un DataFrame.
sequences_df = pd.DataFrame({
    'Size': [sequence.shape[0] for sequence in sequences],
    'Sequence': sequences,
    'ID': range(1, len(sequences) + 1),  # Asigna un número único a cada secuencia
    'Label': [sequence['Label'].unique()[0] for sequence in sequences],
})

In [60]:
sequences_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15947 entries, 0 to 15946
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Size      15947 non-null  int64 
 1   Sequence  15947 non-null  object
 2   ID        15947 non-null  int64 
 3   Label     15947 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 498.5+ KB


In [61]:
sequences_df.head()

,Size,Sequence,ID,Label
0,94,Dst Port Protocol Timestamp Flow Durati...,1,0
1,64,Dst Port Protocol Timestamp Flow Durati...,2,9
2,1,Dst Port Protocol Timestamp Flow Duratio...,3,0
3,5610,Dst Port Protocol Timestamp Flow Dura...,4,9
4,1,Dst Port Protocol Timestamp Flow Duratio...,5,0


In [62]:
print("Análisis estadístico de las secuencias")
print(f"{sequences_df['Size'].describe()}")

Análisis estadístico de las secuencias
count     15947.000000
mean         45.026212
std        2050.380944
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max      237376.000000
Name: Size, dtype: float64


Obtener media y desviación estándar del número de paquetes.

In [63]:
# Obtener la media de paquetes por secuencia
mean = sequences_df['Size'].mean()

# Obtener la desviación estándar muestral (por defecto ddof=1)
deviation = sequences_df['Size'].std()

print(f"Número promedio de paquetes por secuencia: {mean}")
print(f"Desviación estándar (muestral): {deviation}")

Número promedio de paquetes por secuencia: 45.02621182667586
Desviación estándar (muestral): 2050.3809442995553


Obtener número de secuencias asociadas a cada etiqueta

In [64]:
sequences_df["Label"].value_counts()  # Contar la cantidad de secuencias por etiqueta

Label
6     5026
8     4668
0     3805
1     1548
5      635
3      109
2       73
10      32
7       30
9       18
4        3
Name: count, dtype: int64

## Separar secuencias en entrenamiento, validación y prueba

In [73]:
# train, val y test serán los dataframes de secuencias que usaremos para entrenar al modelo
train, test = train_test_split(sequences_df[['Sequence', 'Label']], test_size=0.2, stratify=sequences_df["Label"])
train, val = train_test_split(train, test_size=0.2, stratify=train["Label"])

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [74]:
print(train.shape)
print(val.shape)
print(test.shape)

(10205, 2)
(2552, 2)
(3190, 2)


In [ ]:
"""# Asignar la columna 'ID' dentro de cada DataFrame en train
for i, sequence in enumerate(train["Sequence"]):
    sequence["ID"] = train.iloc[i]["ID"]
# Asignar la columna 'ID' dentro de cada DataFrame en train
for i, sequence in enumerate(test["Sequence"]):
    sequence["ID"] = test.iloc[i]["ID"]


# Concatenar manteniendo la columna 'ID'
train_concatenado = pd.concat(train["Sequence"].tolist(), ignore_index=True)
# Concatenar manteniendo la columna 'ID'
test_concatenado = pd.concat(test["Sequence"].tolist(), ignore_index=True)"""

In [ ]:
"""# Dividir los DataFrames en características (X) y etiquetas (y)
x_train = train_concatenado.drop(columns=['Label', 'ID'])
y_train = train_concatenado[['Label', 'ID']]
# Crear la nueva columna basada en el mapeo
y_train['Label_Mapped'] = y_train['Label'].map(LABELS)

x_test = test_concatenado.drop(columns=['Label', 'ID'])
y_test = test_concatenado[['Label', 'ID']]
# Crear la nueva columna basada en el mapeo
y_test['Label_Mapped'] = y_test['Label'].map(LABELS)

print(f"Dimensiones de x_train: {x_train.shape}")
print(y_train[['Label']].value_counts()) 
y_train.head() # Muestra los primeros 5 paquetes del dataframe de entrenamiento"""

In [ ]:
# Cambiar etiquetas a su estado original
"""original_labels = le.inverse_transform(sequences[-1]['Label'])
print(original_labels)
print(len(original_labels))"""